In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader
import pandas as pd
from IPython.display import Markdown

from typing import Annotated, Literal
from typing_extensions import TypedDict
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.tools.retriever import create_retriever_tool
from langchain.document_loaders import PyMuPDFLoader, PyPDFLoader
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.documents.base import Document
from langchain_core.vectorstores.base import VectorStoreRetriever
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser
from langchain_core.output_parsers.string import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain_experimental.utilities import PythonREPL
from pydantic import BaseModel, Field
from markitdown import MarkItDown
import dotenv
import os

dotenv.load_dotenv()

True

In [5]:
df = pd.read_csv("./data/titanic.csv")

In [6]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [ ]:
md = MarkItDown()

In [22]:
convert_df = md.convert("./data/titanic.csv")

In [23]:
convert_df = convert_df.text_content

In [25]:
prompt = PromptTemplate.from_template(
    """
    table : {convert}

    당신은 데이터 분석 전문가입니다.
    주어진 table 데이터를 기반으로 데이터 분석에 도움을 주세요.

    query : {query}

    """
    )

In [26]:
llm = ChatOpenAI(model="gpt-4o-mini",
                 temperature=0.,)

In [27]:
chain = prompt | llm

In [28]:
query = "이 데이터의 총 행의 갯수는 몇 개인가요?"

In [29]:
result = chain.invoke({"convert":convert_df,
                       "query" : query
                       })

In [31]:
result.content

'주어진 데이터의 총 행의 갯수는 891개입니다.'

In [32]:
def chat_llm(query):
    result = chain.invoke({"convert":convert_df,
                       "query" : query
                       })    
    
    return result.content

In [37]:
a = chat_llm("이 데이터에서 얻을 수 있는 인사이트는 무엇인가요?")

In [39]:
print(a)

주어진 데이터는 타이타닉 승객에 대한 정보로, 여러 가지 인사이트를 도출할 수 있습니다. 다음은 이 데이터를 기반으로 얻을 수 있는 몇 가지 인사이트입니다:

1. **생존율 분석**:
   - 전체 승객 중 생존한 비율과 사망한 비율을 계산하여 생존율을 분석할 수 있습니다.
   - 성별, 나이, 선실 등급(Pclass)별로 생존율을 비교하여 어떤 그룹이 더 높은 생존율을 보였는지 확인할 수 있습니다.

2. **성별에 따른 생존율**:
   - 남성과 여성의 생존율을 비교하여 성별이 생존에 미친 영향을 분석할 수 있습니다. 일반적으로 여성의 생존율이 남성보다 높았던 것으로 알려져 있습니다.

3. **연령대별 생존율**:
   - 연령대별로 생존율을 분석하여 어린이, 청소년, 성인, 노인 등 각 연령대의 생존율을 비교할 수 있습니다. 예를 들어, 어린이의 생존율이 높았는지 확인할 수 있습니다.

4. **선실 등급(Pclass) 분석**:
   - 선실 등급별로 생존율을 분석하여 고급 선실(Pclass 1) 승객이 더 높은 생존율을 보였는지 확인할 수 있습니다. 이는 사회적 지위가 생존에 미친 영향을 나타낼 수 있습니다.

5. **가족 구성원 수에 따른 생존율**:
   - Sibling(형제/자매) 및 Parch(부모/자녀) 수에 따라 생존율을 분석하여 가족과 함께 탑승한 승객이 생존에 유리했는지 확인할 수 있습니다.

6. **요금(Fare) 분석**:
   - 요금 지불 금액에 따른 생존율을 분석하여 높은 요금을 지불한 승객이 생존할 확률이 더 높았는지 확인할 수 있습니다.

7. **탑승 항구(Embarked) 분석**:
   - 탑승 항구별로 생존율을 분석하여 어떤 항구에서 탑승한 승객이 더 높은 생존율을 보였는지 확인할 수 있습니다.

8. **결혼 여부 및 가족 관계**:
   - 승객의 이름에서 결혼 여부를 추정하여 결혼한 여성과 미혼 여성의 생존율을 비교할 수 있습니다.

9. **결측치 분석**:
   - 나이(Age)와 같은 중요한 변

In [67]:
dfccs = pd.read_csv("./data/cookie_cats.csv")

In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  bool  
 4   retention_7     90189 non-null  bool  
dtypes: bool(2), int64(2), object(1)
memory usage: 2.2+ MB


In [68]:
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.agents.agent_types import AgentType
import seaborn as sns

sns.set_style("white")

agent = create_pandas_dataframe_agent(
    ChatOpenAI(model="gpt-4o-mini", temperature=0),
    dfccs,
    verbose=False,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    allow_dangerous_code=True,
)

In [74]:
str(agent)

"verbose=False agent=RunnableAgent(runnable=RunnableAssign(mapper={\n  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_function_messages(x['intermediate_steps']))\n})\n| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'agent_scratchpad': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langc

In [64]:
streaming = agent.stream({"input":query})

In [65]:
for step in streaming:
    print(step)

{'actions': [AgentActionMessageLog(tool='python_repl_ast', tool_input={'query': 'len(df)'}, log="\nInvoking: `python_repl_ast` with `{'query': 'len(df)'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'function_call': {'arguments': '{"query":"len(df)"}', 'name': 'python_repl_ast'}}, response_metadata={'finish_reason': 'function_call', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306'}, id='run-f9ac2467-5137-4137-93f9-61982f58a597')])], 'messages': [AIMessageChunk(content='', additional_kwargs={'function_call': {'arguments': '{"query":"len(df)"}', 'name': 'python_repl_ast'}}, response_metadata={'finish_reason': 'function_call', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306'}, id='run-f9ac2467-5137-4137-93f9-61982f58a597')]}
{'steps': [AgentStep(action=AgentActionMessageLog(tool='python_repl_ast', tool_input={'query': 'len(df)'}, log="\nInvoking: `python_repl_ast` with `{'query': 'len(df)'}`\n\n\n", message_l

In [66]:
print(step["output"])

이 데이터프레임의 총 행의 갯수는 90,189개입니다.
